In [6]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
# from sklearn.cross_validation import ShuffleSplit
from sklearn.model_selection import ShuffleSplit

# Import supplementary visualizations code visuals.py
# import visuals as vs

# Pretty display for notebooks
%matplotlib inline

# Load the Boston housing dataset
headers = ["ID","CT","UCSize","UCShape","MA","SECSize","BN","BC","NN","Mitoses","Diagnosis"]
data = pd.read_csv('breast-cancer-wisconsin.csv', names = headers)
print(data.head())

diagnosis = data['Diagnosis']
features = data.drop(['ID','Diagnosis'], axis = 1)
print(features.head())

        ID  CT  UCSize  UCShape  MA  SECSize  BN  BC  NN  Mitoses  Diagnosis
0  1000025   5       1        1   1        2   1   3   1        1          2
1  1002945   5       4        4   5        7  10   3   2        1          2
2  1015425   3       1        1   1        2   2   3   1        1          2
3  1016277   6       8        8   1        3   4   3   7        1          2
4  1017023   4       1        1   3        2   1   3   1        1          2
   CT  UCSize  UCShape  MA  SECSize  BN  BC  NN  Mitoses
0   5       1        1   1        2   1   3   1        1
1   5       4        4   5        7  10   3   2        1
2   3       1        1   1        2   2   3   1        1
3   6       8        8   1        3   4   3   7        1
4   4       1        1   3        2   1   3   1        1


In [4]:
from sklearn.model_selection import train_test_split

# Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(features, diagnosis, test_size=0.25, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=2, random_state=0)
rfc.fit(X_train, y_train)
# y_predict = rfc.predict(X_test)

score = rfc.score(X_test, y_test)
print("score = ", score)

ValueError: could not convert string to float: ?